In [1]:
import mysql.connector
import json
import pandas as pd
import datetime


mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Kaja25802580",
  database="twitter_database"
)

mycursor = mydb.cursor()


In [2]:
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
import nltk
nltk.downloader.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Win10\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [18]:
df_tweets = pd.read_sql("select id, tweet_text, author_id, place_id, like_count, reply_count, retweet_count, created_at  from tweets WHERE  like_count > 200", mydb)

DatabaseError: Execution failed on sql 'select id, tweet_text, author_id, place_id, like_count, reply_count, retweet_count, created_at  from tweets where and created_at between '2020-03-01 00:00:00' and '2020-12-31 00:00:00'  group by created_at ORDER BY like_count DESC LIMIT 5000': 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'and created_at between '2020-03-01 00:00:00' and '2020-12-31 00:00:00'  group by' at line 1

In [5]:
df_tweets['created_at'] = pd.to_datetime(df_tweets['created_at']).dt.date
datetime.date(2020, 1, 5) in df_tweets['created_at'].values

False

In [6]:
df_tweets

,id,tweet_text,author_id,place_id,like_count,reply_count,retweet_count,created_at
0,1272865255782338564,I’ll be damned if when a COVID-19 vaccine is d...,29442313,None,188708,1973,30254,2020-06-16
1,1347195835092754432,If the COVID vaccine was administered in Chik-...,1141777054347644929,None,152349,163,14146,2021-01-07
2,1239217522093764608,This is depraved. The German government confir...,39279821,None,146988,4233,67532,2020-03-15
3,1353385916107251714,Y'all are clearly more scared of the vaccine t...,2405640916,None,128320,467,23283,2021-01-24
4,1312025841396846597,What if...Trump is lying that he got Covid so ...,1223345935,None,122073,550,26260,2020-10-02
...,...,...,...,...,...,...,...,...
4995,1345353219891933184,Americans who strongly embrace Christian natio...,2550422984,None,250,14,136,2021-01-02
4996,1346108052978601984,As the leader of party in power @narendramodi ...,2681593896,None,250,12,70,2021-01-04
4997,1354776007857876992,#EU asks #AstraZeneca : \ndoes signing a contr...,1347250338,None,250,21,80,2021-01-28
4998,1246098933383696387,University of Pittsburgh develops potential CO...,1209936918,None,249,24,116,2020-04-03


In [7]:
# df_tweets['pos'] = df_tweets['tweet_text'].apply(lambda x:SentimentIntensityAnalyzer().polarity_scores(x)['pos'])
# df_tweets['neg'] = df_tweets['tweet_text'].apply(lambda x:SentimentIntensityAnalyzer().polarity_scores(x)['neg'])
# df_tweets['neu'] = df_tweets['tweet_text'].apply(lambda x:SentimentIntensityAnalyzer().polarity_scores(x)['neu'])
df_tweets['compound'] = df_tweets['tweet_text'].apply(lambda x:SentimentIntensityAnalyzer().polarity_scores(x)['compound'])

In [8]:
df_tweets['label'] = 0
# -1 -> negative
# 0 -> neutral
# 1 -> positive
for ind in df_tweets.index:
    if df_tweets['compound'][ind] < -0.05:
        df_tweets['label'][ind] = -1
    elif df_tweets['compound'][ind] > 0.05:
        df_tweets['label'][ind] = 1
    else:
        df_tweets['label'][ind] = 0

<ipython-input-8-8b01ffdb583c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['label'][ind] = -1
<ipython-input-8-8b01ffdb583c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['label'][ind] = 0
<ipython-input-8-8b01ffdb583c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['label'][ind] = 1


In [9]:
# zliczanie danych
sentiment_df_tweets = pd.DataFrame(df_tweets.groupby('label').size())
sentiment_df_tweets

,0
label,
-1,1500
0,1113
1,2387


In [10]:
import matplotlib.pyplot as plt
neg = int(sentiment_df_tweets.values[0])
neu = int(sentiment_df_tweets.values[1])
pos = int(sentiment_df_tweets.values[2])
sentiment_names = ['Negatywne', 'Neutralne', 'Pozytywne']
sen_tweet_count = [neg, neu, pos]

plt.figure(figsize=(15,10))
plt.rcParams['font.size'] = '20' 
rects = plt.bar(sentiment_names, sen_tweet_count, width=0.6, color=['red', 'yellow', 'green'])
plt.bar_label(rects)
plt.ylabel('Liczba tweetów o określonym sentymencie')
plt.title('Analiza sentymentu dla najbardziej popularnych tweetów')
plt.savefig(f'charts/Sentiment_analysis.png')
plt.close()

In [11]:
sentiment_dict = {
    -1 : {
        'display_name' : 'Negatywne',
        'color' : 'r'
    },
    0 : {
        'display_name' : 'Neutralne',
        'color' : 'y'
    },
    1 : {
        'display_name' : 'Pozytywne',
        'color' : 'g'
    }
}

In [12]:
import matplotlib.pyplot as plt
import numpy as np
import datetime

def plotSentimentInTime_like(sentiment_name):
    new = df_tweets.query('label==@sentiment_name').groupby(['created_at', 'label']).size()
    data = []
    dates = []

    for x in new.index:
        dates.append(x[0])
    for i in new.values:
        data.append(i)
        

    sentiment = sentiment_dict[sentiment_name]
    print('Plotting', sentiment['display_name'])
    plt.plot(dates, data, sentiment['color'] + 'o--', label=sentiment['display_name'])
    return dates[::20]

In [13]:
plt.figure(figsize=(24,10))

dates = None
for sentiment_name in sentiment_dict.keys():
    dates = plotSentimentInTime_like(sentiment_name)
plt.xticks(dates)

plt.ylabel('Liczba tweetów', fontsize='xx-large')
plt.xlabel('Data', fontsize='xx-large')
plt.title('Liczba tweetów o określonym sentymencie dla najbardziej likeownych tweetów')
plt.legend(fontsize='xx-large')
ax = plt.gca()
ax.tick_params(axis="x", labelsize=16, labelrotation=90)
ax.tick_params(axis="y", labelsize=14)
plt.tight_layout()
plt.savefig(f'charts/SenAnalysisInTime.png')
plt.close()

Plotting Negatywne
Plotting Neutralne
Plotting Pozytywne
